In [1]:
from lib.ekyn import *
from torch.utils.data import DataLoader,ConcatDataset,TensorDataset
from lib.datasets import EpochedDataset
from sklearn.model_selection import train_test_split
import torch
from lib.models import RegNetX
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import nn
from torch.nn.functional import relu

CONFIG = {
    'BATCH_SIZE':256,
    'WINDOW_SIZE':5000,
    'LEARNING_RATE':3e-3,
    'TEST_SIZE':.25,
    'DEVICE':'cuda',
    'DEPTHI':[1,1,1,1],
    'WIDTHI':[32,64,128,256],
    'STEM_KERNEL_SIZE':7
}

train_idx,test_idx = train_test_split(get_ekyn_ids(),test_size=.25,random_state=0)
print(train_idx,test_idx)
trainloader = DataLoader(ConcatDataset([EpochedDataset(idx=idx,condition=condition) for idx in train_idx[:2] for condition in ['Vehicle','PF']]),batch_size=CONFIG['BATCH_SIZE'],shuffle=True)
devloader = DataLoader(ConcatDataset([EpochedDataset(idx=idx,condition=condition) for idx in test_idx for condition in ['Vehicle','PF']]),batch_size=CONFIG['BATCH_SIZE'],shuffle=True)

['F1-0', 'B3-1', 'A4-0', 'F1-1', 'E2-1', 'C4-1', 'F5-1', 'E4-0', 'B1-0', 'A1-0', 'C1-0', 'E4-1'] ['A1-1', 'C4-0', 'D1-0', 'E1-0']


In [2]:
from lib.models import YBlock,RegNetY
import math
model = RegNetY(depth=CONFIG['DEPTHI'],width=CONFIG['WIDTHI'],stem_kernel_size=CONFIG['STEM_KERNEL_SIZE'])
# criterion = torch.nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss(weight=torch.tensor([18.3846,  2.2810,  1.9716]))
optimizer = torch.optim.AdamW(model.parameters(),lr=CONFIG['LEARNING_RATE'])
model.to(CONFIG['DEVICE']);
criterion.to(CONFIG['DEVICE']);
print(sum([p.flatten().size()[0] for p in list(model.parameters())]))

44835


In [3]:
lossi = []
trainlossi = []
trainf1 = []
devlossi = []
devf1 = []
model.train()
from lib.utils import training_loop,development_loop
for i in tqdm(range(1000)):
    loss,f1 = training_loop(model=model,trainloader=trainloader,criterion=criterion,optimizer=optimizer,device=CONFIG['DEVICE'])
    trainlossi.append(loss)
    trainf1.append(f1)

    loss,f1 = development_loop(model=model,devloader=devloader,criterion=criterion,device=CONFIG['DEVICE'])
    devlossi.append(loss)
    devf1.append(f1)

    fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,4))
    ax[0].plot(trainlossi)
    ax[0].plot(devlossi)
    ax[1].plot(trainf1)
    ax[1].plot(devf1)
    plt.savefig('loss.jpg')
    plt.close()

  1%|          | 8/1000 [00:52<1:48:36,  6.57s/it]


KeyboardInterrupt: 

In [ ]:
from lib.utils import evaluate
evaluate(devloader,model,criterion,CONFIG['DEVICE'])